In [1]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
import torch
import pywt
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def csv_to_df(file_path):
    # Read the csv file
    df_in = pd.read_csv(file_path)

    # Get the header row
    header_row = list(df_in.columns)

    # Convert the header row to integer values
    header_row = list(map(int, header_row))

    # Create a DataFrame from the header row
    header_df = pd.DataFrame([header_row], columns=df_in.columns)

    # Concatenate the header row with the original DataFrame
    df_imm = pd.concat([header_df, df_in], ignore_index=True)

    # Reset column names
    df_imm.columns = range(df_imm.shape[1])

    # Remove the first column
    df_imm = df_imm.drop(0, axis=1)

    df_out = df_imm.T
    
    return df_out

In [3]:
def z_score(epoch):    
    # Apply z-score normalization to each channel, saved in epoch
    for i in range(epoch.shape[0]):
        channel_epoch = epoch.iloc[i]
        mean = np.mean(channel_epoch)
        std = np.std(channel_epoch)
        z_scored_epoch = (channel_epoch - mean) / std
        epoch.iloc[i] = z_scored_epoch
    
    return epoch

In [4]:
def frequency_to_scale(freq, wavelet='morl', sampling_rate=250):
    # For the Morlet wavelet, scales are inversely proportional to frequency
    center_freq = pywt.central_frequency(wavelet)
    return center_freq / (freq / sampling_rate)

In [5]:
def apply_wavelet_transform(data_norm, wavelet='morl', freq_range=(8, 30), sampling_rate=250):
    """
    Apply wavelet transform to EEG data.
    
    Parameters:
    data_norm (ndarray): 2D array with shape (channels, time_points)
    wavelet (str): Wavelet type (default 'morl')
    freq_range (tuple): Frequency range for the CWT (default (8, 30) Hz)
    sampling_rate (int): Sampling rate of the EEG data (default 250 Hz)
    
    Returns:
    ndarray: 3D array with shape (channels, scales, time_points)
    """
    n_channels, n_times = data_norm.shape
    # Define scales based on the desired frequency range
    scales = frequency_to_scale(np.arange(freq_range[0], freq_range[1]+1), wavelet=wavelet, sampling_rate=sampling_rate)
    
    coeffs = []
    for i in range(n_channels):
        # Compute the wavelet transform coefficients
        coef, _ = pywt.cwt(data_norm.iloc[i], scales=scales, wavelet=wavelet)
        coeffs.append(coef)
    
    # Stack coefficients to form a 3D tensor
    coeffs_done = np.stack(coeffs, axis=0)
    
    return coeffs_done

In [6]:
def apply_bandpass_filter(signal, b, a):
    return filtfilt(b, a, signal)

In [7]:
def processing_livestreamed_signal(file_path):
    # Load the data, in dataframe format
    data_raw = csv_to_df(file_path)
    
    # Filter parameters
    l_freq = 8
    h_freq = 30
    order = 5
    fs = 250

    nyquist = 0.5 * fs
    low = l_freq / nyquist
    high = h_freq / nyquist

    # Calculate filter coefficients
    b, a = butter(order, [low, high], btype='band')
    
    # Apply the filter to each row independently
    for i in range(data_raw.shape[0]):
        data_raw.iloc[i] = apply_bandpass_filter(data_raw.iloc[i], b, a)
        
    # Convert filtered data back to DataFrame
    data_filtered = pd.DataFrame(data_raw)
    
    # Split data into segments of 0.5 seconds, for epoching
    epoch_size = int(fs * 0.5)  # Number of samples in 0.5 seconds
    epochs = [data_filtered.iloc[:, i:i+epoch_size] for i in range(0, len(data_filtered.columns), epoch_size)]
    
    # Create an empty list to store the transformed epochs
    transformed_epochs = []
    
    # Z-score normalization and wavelet transform for each segment and stack them into a tensor (4D array), 
    # Loop through each epoch
    for epoch in epochs:
        # Z-score each epoch
        epoch_norm = z_score(epoch)
        
        # Apply wavelet transformation
        wavelet_tensor = apply_wavelet_transform(epoch_norm)
        
        # Append the transformed epoch to the list
        transformed_epochs.append(wavelet_tensor)
        
        
    # Convert the list of epochs into a tensor dataset
    livestreamed_tensor = torch.tensor(transformed_epochs, dtype=torch.float32)  # Ensure the tensor is float32
     
    return livestreamed_tensor

In [8]:
input_file_path = r"C:\School\EE_Y3\Q4\BAP\eeg_thesis_cnn_repo\data\vue\lefts_adnane.csv"
data_to_predict = processing_livestreamed_signal(input_file_path)

C:\Users\chakr\AppData\Local\Temp\ipykernel_9672\1123883349.py:46: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  livestreamed_tensor = torch.tensor(transformed_epochs, dtype=torch.float32)  # Ensure the tensor is float32


In [10]:
data_to_predict.shape

torch.Size([50, 8, 23, 125])